In [1]:
import os

import openl3
import soundfile as sf

import numpy as np

from pathlib import Path

import tqdm

Using TensorFlow backend.
/home/prs392/miniconda3/envs/audio_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/prs392/miniconda3/envs/audio_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/prs392/miniconda3/envs/audio_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pr

In [2]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [ ]:
import os

import openl3
import soundfile as sf

import numpy as np
import tqdm

from pathlib import Path

root_dir = '/scratch/prs392/incubator/data/OpenMic'

kinds = ['train', 'val', 'test']

for kind in kinds:

    data_dir = os.path.join(root_dir, kind)

    list_of_file_paths = []
    for root, dirs, files in os.walk(data_dir):
        path = root.split(os.sep)
        for file in files:
            if file.endswith(".ogg"):
    #             print(os.path.basename(os.path.join(root, file)))
                list_of_file_paths.append(os.path.join(root, file))

    data_write_dir = os.path.join(root_dir, kind, 'embeddings_6144')
    
    Path(data_write_dir).mkdir(parents=True, exist_ok=True)
#     print(data_write_dir)
#     break

    list_of_npy_files = []
    for root, dirs, files in os.walk(data_write_dir):
        path = root.split(os.sep)
        for file in files:
            if file.endswith(".npy"):
    #             print(os.path.basename(os.path.join(root, file)))
                list_of_npy_files.append(file.split('.')[0])

    # print(list_of_npy_files)
    model = openl3.models.load_audio_embedding_model(input_repr="mel128", content_type="music", embedding_size=6144)

    for audio_file_path in tqdm.tqdm(list_of_file_paths):

        file_name = os.path.basename(audio_file_path)

        file_id = file_name.split('.')[0]

        if file_id not in list_of_npy_files:

            audio, sr  = sf.read(audio_file_path)

    #         emb, ts = openl3.get_audio_embedding(audio, sr, center = False)
            emb, ts = openl3.get_audio_embedding(audio, sr, center = False, model=model, verbose=False)
            new_emb_file_name = file_id + ".npy"
            new_emb_file_path = os.path.join(data_write_dir, new_emb_file_name)

            with open(new_emb_file_path, 'wb+') as f:
                np.save(f, emb)
        else:
            print(file_id + '.npy already exists') 

 96%|█████████▌| 2429/2543 [39:01<01:51,  1.02it/s]